In [1]:
%matplotlib notebook
import os
import os.path as path
import numpy as np
from scipy.optimize import curve_fit
from scipy import interpolate
from sklearn.preprocessing import MinMaxScaler
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import math 
from ipywidgets import interact, interactive, fixed, interact_manual, widgets
import h5py

from pyccapt.calibration.calibration_tools import tools, data_tools, variables, tof_calibration, data_loadcrop
from pyccapt.calibration.calibration_tools import widgets as wd


# The initial value for flight path length, which is the distance of local electrod to detector 
flightPathLength = 110 # mm 
# The initial value for t_0
t0 = 51.74 # ns
variables.init()



In [2]:
number_sample = 300
save_fig=False
plot_fig=True 

In [ ]:
# dataset name
# AL_data_b
# OLO_AL_6_data
# OLO_W_6_data
# OLO_Ni_8_data
# X6Cr17_2V30Min_5_data
# data_1029_Jun-27-2022_15-53_Pd
# data_115_Jul-27-2022_17-44_Powersweep3

In [ ]:
tdc, dataset = widgets.dataset_tdc_selection()
display(tdc, dataset)

$$\textbf{You can specify which dataset to use in below block}$$

In [3]:
tdc_model = tdc.value

dataset_name = dataset.value

p = path.abspath(path.join("", "../../../.."))

variables.init()

path_main = os.path.join(p, 'tests//data')
filename_main = path_main + '//' + dataset_name + '.h5'

variables.path = os.path.join(p, 'tests//results//load_crop')
variables.result_path = os.path.join(p, 'tests/results/ion_type_selection/' + dataset_name)
if not os.path.isdir(variables.result_path):
        os.makedirs(variables.result_path, mode=0o777, exist_ok=True)
        
filename = variables.path + '//' + dataset_name + '//' + dataset_name + '_cropped' + '.h5'



head, tail = os.path.split(filename)
figname = os.path.splitext(tail)[0]

data = data_tools.read_hdf5_through_pandas(filename)


dld_highVoltage = data['dld/high_voltage'].to_numpy()
dld_pulseVoltage = data['dld/pulse_voltage'].to_numpy()
dld_t = data['dld/t'].to_numpy()
dld_x = data['dld/x'].to_numpy()
dld_y = data['dld/y'].to_numpy()

In [4]:
data

,dld/high_voltage,dld/pulse_voltage,dld/start_counter,dld/t,dld/x,dld/y
0,5267.498047,5267.498047,55251.0,622.754406,3.279184,4.170612
1,5267.498047,5267.498047,55287.0,638.987292,21.203265,-21.457959
2,5267.498047,5267.498047,55309.0,623.385342,5.953469,6.813061
3,5267.498047,5267.498047,56063.0,628.576848,17.955918,-11.620408
4,5267.498047,5267.498047,56440.0,634.749048,-21.903673,0.159184
...,...,...,...,...,...,...
1882844,5712.204590,5712.204590,43305.0,601.762068,2.069388,-6.335510
1882845,5712.204590,5712.204590,43628.0,621.602262,29.130612,16.937143
1882846,5712.204590,5712.204590,44204.0,604.141794,6.240000,-5.762449
1882847,5712.204590,5712.204590,44353.0,602.152974,1.146122,6.685714


In [20]:
with h5py.File(filename_main, 'r') as hdf:
    mc = np.array(hdf["cropped_dld/mc"])

In [21]:
def mc_his(bin_size, prominence, distance, percent, plot, figname, mc, mc_ideal):

    bin_size = float(bin_size)
    prominence = int(prominence)
    distance = int(distance)
    percent = int(percent)
    if mc_ideal == True:
        mc_ideal = np.array(variables.listMaterial)
        print('List of mc are:', mc_ideal)
    else:
        mc_ideal = np.zeros(0)
    max_hist, left_right_peaks, peaks_sides, max_paek_edges, index_max = tools.massSpecPlot(mc[mc < 100], bin_size, mc_ideal, distance=distance, percent=percent, prominence=prominence, plot=plot, label='tof', fig_name=figname)
    mrp = (max_hist / (left_right_peaks[1] - left_right_peaks[0]))
    print('Mass resolving power for the highest peak (MRP --> m/m_2-m_1):', mrp)
    for i in range(len(peaks_sides)):
        print('Peaks ', i, 'is: {:.2f}'.format(peaks_sides[i,0]), 'peak window sides are: {:.2f} - {:.2f}'.format(peaks_sides[i,2], peaks_sides[i,3]))
    return peaks_sides

In [22]:
peaks_sides = interact_manual(mc_his, bin_size=widgets.Textarea(value="0.1")
               ,prominence=widgets.Textarea(value="60"), distance=widgets.Textarea(value="100")
               ,percent=widgets.Textarea(value="20"), plot=fixed(True), mc=fixed(mc),  figname=fixed(figname)
                , mc_ideal=[('False',False),('True',True)]);



interactive(children=(Textarea(value='0.1', description='bin_size'), Textarea(value='60', description='promine…

In [8]:
isotopeTableFile = '../../../../files/isotopeTable.h5'
dataframe = data_tools.read_hdf5_through_pandas(isotopeTableFile)
elementsList = dataframe['element']
elementIsotopeList = dataframe['isotope']
elementMassList =  dataframe['weight']
abundanceList = dataframe['abundance']

elements = list(zip(elementsList,elementIsotopeList,elementMassList,abundanceList))
dropdownList = []
for element in elements:
    tupleElement = ("{} ({}) ({:.2f})".format(element[0],element[1],element[3]),element[2],)
    dropdownList.append(tupleElement)

chargeList = [(1,1,),(2,2,),(3,3,),(4,4,)]
dropdown = wd.dropdownWidget(dropdownList,"Elements")
dropdown.observe(wd.on_change)


chargeDropdown = wd.dropdownWidget(chargeList,"Charge")
chargeDropdown.observe(wd.on_change_charge)

wd.compute_element_isotope_values_according_to_selected_charge()

buttonAdd = wd.buttonWidget("ADD")
buttonDelete = wd.buttonWidget("DELETE")
buttonReset = wd.buttonWidget("RESET")

display(dropdown)
display(chargeDropdown)
display(buttonAdd)
display(buttonDelete)
display(buttonReset)
listMaterial = buttonAdd.on_click(wd.onClickAdd)
buttonDelete.on_click(wd.onClickDelete)
buttonReset.on_click(wd.onClickReset)

Dropdown(description='Elements', options=(('H (1) (99.98)', 1.01), ('H (2) (0.01)', 2.01), ('He (3) (0.00)', 3…

Dropdown(description='Charge', options=((1, 1), (2, 2), (3, 3), (4, 4)), value=1)

Button(description='ADD', icon='check', style=ButtonStyle(), tooltip='ADD')

Button(description='DELETE', icon='check', style=ButtonStyle(), tooltip='DELETE')

Button(description='RESET', icon='check', style=ButtonStyle(), tooltip='RESET')

In [ ]:
# add label to to hdf file
data_loadcrop.add_tof_mc_data_to_hdf5(filename_main, dld_t_calib_final, mc_p_1)
data_loadcrop.add_tof_mc_data_to_hdf5(filename, dld_t_calib_final, mc_p_1)
